# Описание/Пошаговая инструкция выполнения домашнего задания:
В качестве данных выберете возьмите датасет RuCoLA для русского языка https://github.com/RussianNLP/RuCoLA (в качестве train возьмите in_domain_train.csv, а в качестве теста in_domain_dev.csv).

Разбейте in_domain_train на train и val.

Зафайнтьюньте и протестируйте RuBert или RuRoBerta на данной задаче (можно взять любую предобученную модель руберт с сайта huggingface. Например, ruBert-base/large https://huggingface.co/sberbank-ai/ruBert-base / https://huggingface.co/sberbank-ai/ruBert-large или rubert-base-cased https://huggingface.co/DeepPavlov/rubert-base-cased, ruRoberta-large https://huggingface.co/sberbank-ai/ruRoberta-large, xlm-roberta-base https://huggingface.co/xlm-roberta-base).


Возьмите RuGPT3 base или large и решите данное задание с помощью методов few-/zero-shot.

а) переберите несколько вариантов затравок;

б) протестируйте различное число few-shot примеров (0, 1, 2, 4).

Обучите и протестируйте модель RuT5 на данной задаче (пример finetun’а можете найти здесь https://github.com/RussianNLP/RuCoLA/blob/main/baselines/finetune_t5.py).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train_url = 'https://raw.githubusercontent.com/RussianNLP/RuCoLA/main/data/in_domain_train.csv'
dev_url = 'https://raw.githubusercontent.com/RussianNLP/RuCoLA/main/data/in_domain_dev.csv'

train_df = pd.read_csv(train_url)
test_df = pd.read_csv(dev_url)


In [3]:
train_df.head()

,id,sentence,acceptable,error_type,detailed_source
0,0,"Вдруг решетка беззвучно поехала в сторону, и н...",1,0,Paducheva2004
1,1,Этим летом не никуда ездили.,0,Syntax,Rusgram
2,2,Только Иван выразил какую бы то ни было готовн...,1,0,Paducheva2013
3,3,"Теперь ты видишь собственными глазами, как тут...",1,0,Paducheva2010
4,4,На поверку вся теория оказалась полной чепухой.,1,0,Paducheva2010


In [4]:
test_df.head()

,id,sentence,acceptable,error_type,detailed_source
0,0,Иван вчера не позвонил.,1,0,Paducheva2013
1,1,"У многих туристов, кто посещают Кемер весной, ...",0,Syntax,USE8
2,2,Лесные запахи набегали волнами; в них смешалос...,1,0,USE5
3,3,Вчера президент имел неофициальную беседу с ан...,1,0,Seliverstova
4,4,Коллега так и не признал вину за катастрофу пе...,1,0,Testelets


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7869 entries, 0 to 7868
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               7869 non-null   int64 
 1   sentence         7869 non-null   object
 2   acceptable       7869 non-null   int64 
 3   error_type       7869 non-null   object
 4   detailed_source  7869 non-null   object
dtypes: int64(2), object(3)
memory usage: 307.5+ KB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983 entries, 0 to 982
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               983 non-null    int64 
 1   sentence         983 non-null    object
 2   acceptable       983 non-null    int64 
 3   error_type       983 non-null    object
 4   detailed_source  983 non-null    object
dtypes: int64(2), object(3)
memory usage: 38.5+ KB


In [3]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['acceptable'], random_state=42)
train_df.shape, val_df.shape


((6295, 5), (1574, 5))

In [4]:
from datasets import Dataset


In [9]:
train_dataset = Dataset.from_pandas(train_df[['sentence', 'acceptable']])
val_dataset   = Dataset.from_pandas(val_df[['sentence', 'acceptable']])
test_dataset  = Dataset.from_pandas(test_df[['sentence', 'acceptable']])

# RuBERT

In [ ]:
from transformers import AutoTokenizer

model_name = "ai-forever/ruBert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset   = val_dataset.map(tokenize_function, batched=True)
test_dataset  = test_dataset.map(tokenize_function, batched=True)

In [11]:
train_dataset = train_dataset.rename_column("acceptable", "labels")
val_dataset   = val_dataset.rename_column("acceptable", "labels")
test_dataset  = test_dataset.rename_column("acceptable", "labels")

In [12]:
for ds in [train_dataset, val_dataset, test_dataset]:
    for col in list(ds.features):
        if col not in ["input_ids", "attention_mask", "labels", "token_type_ids"]:
            ds = ds.remove_columns(col)

In [ ]:
cols_to_remove = ["sentence", "__index_level_0__"]

train_dataset = train_dataset.remove_columns(cols_to_remove)
val_dataset   = val_dataset.remove_columns(cols_to_remove)
test_dataset  = test_dataset.remove_columns(cols_to_remove)


In [14]:
print(train_dataset[0])
print(train_dataset.features)


{'labels': 1, 'input_ids': [101, 142, 5962, 121, 5141, 49850, 6652, 15520, 672, 9095, 126, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from transformers import AutoModelForSequenceClassification

model_rubert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import matthews_corrcoef

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "matthews_correlation": matthews_corrcoef(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="./rubert_rucola",
    eval_strategy="epoch",              
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="matthews_correlation",
    greater_is_better=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_strategy="steps",          
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model_rubert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

test_metrics = trainer.evaluate(test_dataset)
print(test_metrics)


In [ ]:
trainer.train()


In [22]:
test_metrics_rubert = trainer.evaluate(test_dataset)
test_metrics_rubert

/Users/vadim/Code/OTUS/ML-Prof/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.716770350933075,
 'eval_accuracy': 0.8006103763987793,
 'eval_runtime': 7.3254,
 'eval_samples_per_second': 134.191,
 'eval_steps_per_second': 4.232,
 'epoch': 10.0}

# GPT

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "ai-forever/rugpt3large_based_on_gpt2"  # или локальный путь

tokenizer_gpt = AutoTokenizer.from_pretrained(model_name)
model_gpt = AutoModelForCausalLM.from_pretrained(model_name)

generator = pipeline(
    "text-generation",
    model=model_gpt,
    tokenizer=tokenizer_gpt,
    do_sample=False, 
    max_new_tokens=3,
    temperature=0.0,
    max_length=None
)


In [ ]:
prompts = [ 
    "Грамматически ли правильно это русское предложение?\n"
    "Ответь на следующей строке одним словом: да или нет.\n\n",

    "Тебе даётся одно русское предложение.\n"
    "Определи, грамматически ли оно корректно.\n"
    "Отвечай строго одним словом: 'да' (если предложение корректно) "
    "или 'нет' (если предложение некорректно).\n"
    "Никаких других слов, комментариев и знаков препинания после ответа.\n\n",

    "Ты выступаешь в роли эксперта-лингвиста по русскому языку.\n"
    "Тебе последовательно показывают отдельные русские предложения.\n"
    "Для каждого предложения нужно решить задачу лингвистической приемлемости:\n"
    "- если предложение грамматически корректно и звучит естественно для носителя русского языка,\n"
    "  считаем его приемлемым и отвечаем 'да';\n"
    "- если предложение содержит грамматическую ошибку или звучит неестественно,\n"
    "  считаем его неприемлемым и отвечаем 'нет'.\n"
    "Ответ должен состоять ровно из одного слова: да или нет, без каких‑либо пояснений.\n\n",
]


In [84]:
def build_prompt(prompt, sentence, examples=None):
    if examples:
        for s, y in examples:
            label = "да" if y == 1 else "нет"
            prompt += f"Предложение: {s}\nОтвет: {label}\n\n"
    prompt += f"Предложение: {sentence}\nТвой ответ: "
    return prompt


In [98]:
def classify_with_rugpt3(prompt, sentence, examples=None):
    answer = build_prompt(prompt,sentence, examples)
    out = generator(answer)[0]["generated_text"]

    # берём всё после последнего "Ответ:"
    answer = out.split("Твой ответ:")[-1].strip().lower()

    if "да" in answer and "нет" not in answer:
        return 1
    if "нет" in answer:
        return 0
    # если модель написала что-то странное — по умолчанию считаем "неприемлемо"
    return 0


In [117]:
for s, y in test_df.sample(5, random_state=42)[["sentence", "acceptable"]].values:
    pred = classify_with_rugpt3(prompts[0],s, examples=None)
    print("S:", s)
    print("gold:", y, "pred:", pred)
    print("-"*40)


S: Том и Гек разговаривают.
gold: 1 pred: 1
----------------------------------------
S: Я воздержался от каких бы то ни было комментариев.
gold: 1 pred: 1
----------------------------------------
S: В доме был большой круглый стой, все любили собираться и разговаривать в гостиной у него.
gold: 0 pred: 1
----------------------------------------
S: Так я и пойду!
gold: 1 pred: 1
----------------------------------------
S: Кто бы ни приезжали в наш городок, все поражались чистоте и опрятности, в какой содержатся дворы, улицы, пристань.
gold: 0 pred: 1
----------------------------------------


In [114]:
def eval_rugpt3_few_shot(prompt,n_shots, k=200):
    """
    n_shots: сколько примеров добавить в промпт (0,1,2,4)
    k: сколько первых примеров теста использовать для оценки
    """
    # берём n_shots примеров из train
    few_shot_examples = None
    if n_shots > 0:
        few_shot_examples = [
            (x, y) for x, y in train_df.sample(n_shots, random_state=42)[['sentence', 'acceptable']].itertuples(index=False)
            ]

    preds = []
    golds = []
    for i in range(k):
        s = test_df.iloc[i]["sentence"]
        y = test_df.iloc[i]["acceptable"]
        pred = classify_with_rugpt3(prompt,s, few_shot_examples)
        preds.append(pred)
        golds.append(y)

    acc = accuracy_score(golds, preds)
    return acc


In [125]:
results = {}
for i, prompt in enumerate(prompts):
    results[i] = {}
    print(f"Prompt {i}:")
    for shots in [0, 1, 2, 4]:
        acc = eval_rugpt3_few_shot(prompt,n_shots=shots, k=983)  
        results[i][shots] = acc
        print(f"{shots}-shot accuracy: {acc:.3f}")
    print("=====================")


Prompt 0:
0-shot accuracy: 0.740
1-shot accuracy: 0.744
2-shot accuracy: 0.628
4-shot accuracy: 0.650
Prompt 1:
0-shot accuracy: 0.548
1-shot accuracy: 0.734
2-shot accuracy: 0.717
4-shot accuracy: 0.739
Prompt 2:
0-shot accuracy: 0.564
1-shot accuracy: 0.718
2-shot accuracy: 0.714
4-shot accuracy: 0.739


Удивительно, но лучший результат у простого промпта с 1 примером или без него

# RuT5

In [ ]:
from functools import partial
from transformers import T5Tokenizer

tokenizer_t5 = T5Tokenizer.from_pretrained("ai-forever/ruT5-large")

In [21]:
POS_LABEL = "yes"
NEG_LABEL = "no"

def preprocess_examples(examples, tokenizer=tokenizer_t5):
    # Токенизируем входные предложения
    result = tokenizer(examples["sentence"], padding=False)

    # Формируем строковые таргеты для RuT5: "yes"/"no"
    if "acceptable" in examples:
        label_sequences = []
        for label in examples["acceptable"]:
            if label == 1:
                target_sequence = POS_LABEL
            elif label == 0:
                target_sequence = NEG_LABEL
            else:
                raise ValueError("Unknown class label")
            label_sequences.append(target_sequence)
    else:
        # на тесте может не быть меток — тогда пустые строки
        label_sequences = ["" for _ in examples["sentence"]]

    # Токенизируем таргеты и кладём в "labels"
    result["labels"] = tokenizer(label_sequences, padding=False)["input_ids"]

    return result


In [23]:
from datasets import Dataset

train_hf = Dataset.from_pandas(train_df[["sentence", "acceptable"]])
val_hf   = Dataset.from_pandas(val_df[["sentence", "acceptable"]])
test_hf  = Dataset.from_pandas(test_df[["sentence", "acceptable"]])



In [ ]:
train_hf = train_hf.remove_columns("__index_level_0__")
val_hf   = val_hf.remove_columns("__index_level_0__")
test_hf  = test_hf.remove_columns("__index_level_0__")

In [25]:
tokenized_train = train_hf.map(
    preprocess_examples,
    batched=True,
)

tokenized_val = val_hf.map(
    preprocess_examples,
    batched=True,
    
)

tokenized_test = test_hf.map(
    preprocess_examples,
    batched=True,
)

Map:   0%|          | 0/6295 [00:00<?, ? examples/s]

Map:   0%|          | 0/1574 [00:00<?, ? examples/s]

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

In [30]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p, tokenizer):
    preds = p.predictions
    string_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    y_pred = np.array([1 if s.strip() == "yes" else 0 for s in string_preds])

    labels = np.where(p.label_ids != -100, p.label_ids, tokenizer.pad_token_id)
    string_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    y_true = np.array([1 if s.strip() == "yes" else 0 for s in string_labels])

    return {
        "accuracy": accuracy_score(y_true, y_pred),
    }

In [ ]:
from transformers import  T5ForConditionalGeneration

model_name = "ai-forever/ruT5-large"  

model_t5 = T5ForConditionalGeneration.from_pretrained(model_name)


In [59]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="rut5_large_rucola",
    eval_strategy="epoch",   
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=1e-5,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.0,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    report_to="none",
)



In [60]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer_t5, pad_to_multiple_of=8)

trainer = Seq2SeqTrainer(
    model=model_t5,
    args=training_args,
    train_dataset=tokenized_train,  
    eval_dataset=tokenized_val,      
    data_collator=data_collator,
    compute_metrics=lambda p: compute_metrics(p, tokenizer),
)


In [ ]:
trainer.train()

In [62]:
test_result = trainer.predict(tokenized_test, max_length=5)
print(test_result.metrics)


/Users/vadim/Code/OTUS/ML-Prof/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'test_loss': 0.35401326417922974, 'test_accuracy': 0.8107833163784334, 'test_runtime': 24.9192, 'test_samples_per_second': 39.447, 'test_steps_per_second': 9.872}


# Итог

Показатели accuracy на тесте.

- ruBERT = 0.80
- лучший GPT = 0.74
- RuT5 = 0.81